In [11]:
import os
import json
import re
import src.notebook.path as path
output_file = path.wiki
root_dir = '../data/text'

# メモリ効率よくファイルを逐次的に処理
with open(output_file, 'w', encoding='utf-8') as json_file:
    #json_file.write('[')  # JSON配列の開始
    first_entry = True

    # root_dir内のすべてのサブディレクトリとファイルを探す
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            #json_file.write('[')
            #first_entry = True

            # ファイルを開いて、内容を処理
            with open(file_path, 'r', encoding='utf-8') as f:
                content = re.sub(r'\b(\w+)\b(?:\n{2,})\1\b', r'\1', f.read())

                content = re.sub(r'\n.*?\.', r'', content, re.MULTILINE)

                content = re.sub(r'\n', '', content)

                # docタグごとにJSONに変換
                docs = re.findall(r'<doc.*?>(.*?)</doc>', content, re.DOTALL)
                for doc in docs:
                    if not first_entry:
                        json_file.write('\n')  # 最初のエントリでない場合はカンマを付ける
                    json.dump({"text": doc.strip()}, json_file, ensure_ascii=False)
                    first_entry = False
            #json_file.write(']')
            #break
    #json_file.write(']')  # JSON配列の終了


In [12]:
import json
import src.notebook.path as path

output_file = path.wiki_batched
with open(output_file, 'w', encoding='utf-8') as json_file:
    stack = ""
    with open(path.wiki, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            if len(data["text"]) <= 10: continue
            elif len(data["text"]) <= 512:
                stack += data["text"] + "<pad>"
                if len(stack) > 512:
                    json.dump({"text": stack}, json_file, ensure_ascii=False)
                    json_file.write('\n')
                    stack = ""
            elif len(data["text"]) > 1024:
                split_index = [0]
                split_num = len(data["text"]) // 1024 + 1
                split_size = len(data["text"]) // split_num
                for i in range(split_num-1):
                    split_index.append(data["text"].rfind("。", split_index[i], split_index[i] + split_size)+1)
                    json.dump({"text": data["text"][split_index[i]:split_index[i+1]]}, json_file, ensure_ascii=False)
                    json_file.write('\n')
                json.dump({"text": data["text"][split_index[-1]:]}, json_file, ensure_ascii=False)
                json_file.write('\n')
            else:
                json.dump({"text": data["text"]}, json_file, ensure_ascii=False)
                json_file.write('\n')
            #break